In [ ]:
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt
import csv
from tqdm import tqdm

## e-butterfly

In [ ]:
buttefly_data = pd.read_csv(r'/ebutterfly/Darwin/0177350-230224095556074/occurrence_.csv', 
                  sep="	|	",
                  engine="python")

In [ ]:
buttefly_data.shape

In [ ]:
buttefly_data["eventID"].value_counts()

In [ ]:
buttefly_data_US = buttefly_data[buttefly_data["countryCode"] == 'US']
buttefly_data_US.reset_index(drop=True, inplace=True)
buttefly_data_US = buttefly_data_US[buttefly_data_US["year"] >= 2010]
buttefly_data_US.reset_index(drop=True, inplace=True)

In [ ]:
buttefly_data_US.to_csv("ebutterfly/Darwin/0177350-230224095556074/occ_usa.csv", index=False)

In [ ]:
buttefly_data_US = pd.read_csv("/ebutterfly/Darwin/0177350-230224095556074/occ_usa.csv")

In [ ]:
buttefly_data_US.shape

'verbatimLocality' ->location
'occurrenceID' -> eb-observation
'eventID' -> eb-checklist
'locationID' -> eb-site
'species'
'decimalLatitude'
'decimalLongitude'
'taxonID' -> eb-species

In [ ]:
buttefly_data_US

In [ ]:
#checklists

In [ ]:
ebutterfly_species_list = buttefly_data_US["taxonID"].unique()

In [ ]:
len(ebutterfly_species_list)

In [ ]:
ebutterfly_species_list

In [ ]:
buttefly_data_US["locationID"].value_counts()

In [ ]:
US_unique_locations = buttefly_data_US["verbatimLocality"].value_counts()

In [ ]:
count = 0
for loc in US_unique_locations.keys():
    if (US_unique_locations[loc] > 1):
        print(US_unique_locations[loc], loc)
        count += 1
print(count)

In [ ]:
#buttefly_data_US.set_index(['verbatimLocality'], inplace=True)

In [ ]:
buttefly_data_US.iloc[buttefly_data_US.index[buttefly_data_US["verbatimLocality"] == "Bartram's Garden"]]['decimalLatitude']

In [ ]:
ebutterfly_checklist_ids = buttefly_data_US["eventID"].unique()

In [ ]:
len(ebutterfly_checklist_ids)

In [ ]:
species = {}
print(len(ebutterfly_species_list))
ebutterfly_species_list = [x for x in ebutterfly_species_list if str(x) != 'nan']
print(len(ebutterfly_species_list))

for i, species_id in enumerate(ebutterfly_species_list):
    species[species_id] = i

In [ ]:
species

In [ ]:
checklists = {}
for checklist_id in tqdm(ebutterfly_checklist_ids):
    species_in_checklist = buttefly_data_US.iloc[buttefly_data_US.index[buttefly_data_US["eventID"] == checklist_id]]["taxonID"].values
#     species_vals = [species.get(key) for key in species_in_checklist]
#     print(species_vals, species_in_checklist)
    checklists["checklist_id"] = species_in_checklist

## e-bird

In [ ]:
ebird_data = pd.read_csv(r'ebird/all_summer_hotspots.csv')

In [ ]:
ebird_data.shape

In [ ]:
ebird_data

In [ ]:
ebird_data.groupby(["hotspot_name"]).size()

In [ ]:
ebird_data["hotspot_name"].value_counts()

In [ ]:
t = np.unique(ebird_data["hotspot_name"].values.tolist(), return_counts=True)

In [ ]:
t[0].shape

In [ ]:
ebird_locations = ebird_data["hotspot_name"].values.tolist()
ebuttefly_locations = buttefly_data_US["verbatimLocality"].values.tolist()
locations_overlap = list(set(ebird_locations) & set(ebuttefly_locations))

In [ ]:
# ebird_data.set_index(['lon', 'lat'], inplace=True)
# buttefly_data_US.set_index(['decimalLongitude', 'decimalLatitude'], inplace=True)

In [ ]:
buttefly_data_US

In [ ]:
ebird_data

In [ ]:
# ebird_locations = ebird_data[("lon", "lat")].values.tolist()
# ebuttefly_locations = buttefly_data_US[("decimalLatitude", "decimalLongitude")].values.tolist()
# locations_overlap = list(set(ebird_locations) & set(ebuttefly_locations))

In [ ]:
locations_overlap

In [ ]:
len(locations_overlap)

### Parse the overlapped locations:

In [ ]:
for loc_id in locations_overlap:
    d1 = ebird_data.iloc[ebird_data.index[ebird_data["hotspot_name"] == loc_id]]
    d2 = buttefly_data_US.iloc[buttefly_data_US.index[buttefly_data_US["verbatimLocality"] == loc_id]]

In [ ]:
df_1 = ebird_data[["lon", "lat"]]
df_2 = buttefly_data_US.drop_duplicates(subset=["decimalLongitude", "decimalLatitude"])

In [ ]:
df_2 = df_2.rename(columns={"decimalLongitude": "lon", "decimalLatitude": "lat"})

In [ ]:
df_1

In [ ]:
df_2

In [ ]:
df_lon_lat = pd.concat([df_1, df_2], axis=0)

In [ ]:
df_lon_lat

In [ ]:
grouped_df = df_lon_lat.groupby(["lon", "lat"])

group_counts = df_lon_lat[["lon", "lat"]].value_counts().rename('count')

sorted_unique_values = group_counts.groupby(level=0).apply(lambda x: x.sort_values(ascending=False)).reset_index()

print(sorted_unique_values[sorted_unique_values["count"] > 1])

In [ ]:
import folium
import pandas as pd
from IPython.display import display

map_center = [37.0902, -95.7129] # USA center coordinates
map_zoom = 4
map_obj = folium.Map(location=map_center, zoom_start=map_zoom, tiles='Stamen Terrain')

#ebird - blue
for index, row in df_1.iterrows():
    lat = row['lat']
    lon = row['lon']
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_obj)

    # ebutterfly - red
for index, row in df_2.iterrows():
    lat = row['lat']
    lon = row['lon']
    folium.CircleMarker(
        location=[lat, lon],
        radius=5,
        color='red',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(map_obj)
    
# Display the map
display(map_obj)